In [2]:
# imports
import pandas as pd
import requests
import json

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [219]:
api_key = 'fsq3H+w/7/Ly8+TEfc4uklcJDOfrsHJe5A2JSP5Al2+k+ww='
location = 'London, United Kingdom'
url = "https://api.foursquare.com/v3/places/search?near=" + location

# Create dictionary for headers
headers = {"Accept": "application/json"}
# Add key with our API KEY
headers['Authorization'] = api_key

result = requests.get(url, headers=headers)


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [222]:
#I need to extract the most
#relevant data in our case. 

instance = london['results']
#1. I will loop through the dictionary and get the information i need
foursquare = []
for item in instance:
    foursquare.append({
        'distance':item['distance'],
        'name':item['name'],
        'location':item['location']
    })

#3. I will parse foursquare into dataframe
foursquaredf = pd.DataFrame(foursquare)
foursquaredf

,distance,name,location
0,193,National Gallery,"{'address': 'Trafalgar Sq', 'admin_region': 'E..."
1,494,WatchHouse,"{'address': '7 Upper St Martins Lane', 'admin_..."
2,617,Picturehouse Central,"{'address': 'Shaftesbury Ave', 'admin_region':..."
3,683,Foyles,"{'address': '113-119 Charing Cross Rd', 'admin..."
4,856,International House London,"{'address': '16 Stukeley St', 'admin_region': ..."
5,873,Big Ben (Elizabeth Tower),"{'address': 'Parliament Sq', 'admin_region': '..."
6,1017,Palace of Westminster,"{'address': 'Houses of Parliament', 'admin_reg..."
7,1156,Scarfes Bar,"{'address': '252 High Holborn', 'admin_region'..."
8,1165,British Museum,"{'address': 'Great Russell St', 'admin_region'..."
9,1219,Dior,"{'address': '160-162 New Bond St', 'admin_regi..."


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [223]:

with open('response.json', 'r') as f:
    yelp = json.loads(f.read())


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [224]:

# Extracting id, name, review_count, and rating from each dictionary in the list
data = []
for business in yelp['businesses']:
    data.append({
        'id':business['id'],
        'name': business['name'],
        'price': business['price'],
        'review_count': business['review_count'],
        'rating': business['rating'],
        'distance': business['distance']
    })




Put your parsed results into a DataFrame

In [225]:
# Converting the list of dictionaries into a DataFrame
yelpd = pd.DataFrame(data, columns = ['name', 'price',
                                     'review_count', 'rating','distance'])

#rename 'name' as 'resto_name' to avoid future confusion
yelpdf= yelpd.rename(columns={'name': 'resto_name'})

# Displaying the DataFrame
yelpdf

,resto_name,price,review_count,rating,distance
0,Dishoom,££,2556,4.5,626.567432
1,Sketch,££££,1031,4.0,388.943719
2,Yauatcha,£££,531,4.0,104.027201
3,Dishoom,£££,729,4.6,262.732936
4,The Mayfair Chippy,££,510,4.5,1205.537933
5,Blacklock,£££,150,4.6,162.359136
6,Kiln,££,82,4.3,170.850239
7,Bob Bob Ricard,££££,190,4.1,111.345078
8,Flat Iron,££,480,4.5,186.272039
9,Social Eating House,£££,99,4.1,86.721346


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp data provides more coverage compared to Foursquare data in this situation. Firstly, that is due to the presence of ratings which will make it easier for customers to relate different POIs in the area. 
Secondly, it provides a varied range of Points of interests and then gives enough quantititive data that would make it easier to make meaningful conclusions when measuring performance. 

Get the top 10 restaurants according to their rating

In [226]:
yelpresto = yelpdf.sort_values(by=['rating'], ascending = False)
bestresto = yelpresto.head(10)

bestresto

,resto_name,price,review_count,rating,distance
17,The Ledbury,££££,173,4.6,4458.536864
3,Dishoom,£££,729,4.6,262.732936
5,Blacklock,£££,150,4.6,162.359136
0,Dishoom,££,2556,4.5,626.567432
8,Flat Iron,££,480,4.5,186.272039
4,The Mayfair Chippy,££,510,4.5,1205.537933
15,Burger & Lobster,££,114,4.5,2066.044802
13,Hawksmoor Seven Dials,£££,398,4.4,715.805011
10,Dinner by Heston Blumenthal,££££,312,4.4,2047.209028
19,Petrus,££££,135,4.4,2065.043045


In [218]:
#for future use, I will export bestresto as excel
bestres= bestresto.to_excel("resto.xlsx", sheet_name="resto")
